# Option pricing with different volatilities
Motivation towards $\mathbb{R}^n \to \mathbb{R}^m$

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/My Drive/Teaching/LT_2021_2022/ST456/Week03')

In [ ]:
from BS_Fun import *
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
k = np.arange(1,100,0.1)
sig = [0.5,1,2,3]
trial = BS_Options_Pricing_sigma(S=20,K=k,T=1,r=0.01,sigma=sig)
trial.make_calls()


In [ ]:
trial.calls.update({'strike':k})
df(trial.calls).plot('strike')

## Non-linear approximation: towards $\mathbb{R}^{16} \to \mathbb{R}$

In [ ]:
from tf_Fun import *
initialisation_tf()

In [ ]:
def generate_sample_no_noise(S=20,K=np.arange(5,21,1),
                             T=1,r=0,true_sigma=np.arange(0.1,5,0.01)):
    '''
    We receive data over different calls, and would like to recover sigma
    Assume that we have no noise in the data
    '''
    trial = BS_Options_Pricing_sigma(S,K,T,r,true_sigma)
    trial.make_calls()
    data=df(trial.calls).transpose()
    data.reset_index(inplace=True)
    data.columns = ['sig']+K.tolist()
    return data

In [ ]:
data = generate_sample_no_noise()
data.set_index('sig').iloc[nr.randint(500,size=10)].transpose().plot(legend=False)

In [ ]:
y = data.pop('sig')
X = data

X_train, X_test, y_train, y_test = skms.train_test_split(X, y, test_size=0.33, random_state=0)

feature_normaliser = layers.Normalization(input_shape=[16], axis=1,name='normaliser')
feature_normaliser.adapt(X_train)

In [ ]:
lr=linear_regression_make(feature_normaliser)

history = lr.fit(
    X_train,
    y_train,
    epochs=10,
    verbose=1,
    validation_data = (X_test, y_test)
)
#Problems! Initialisation matters 

see https://keras.io/api/layers/initializers/ for instance

In [ ]:
lr.weights

**Exercise:** Observe the importance of zeroweights and / or SGD learning rate


In [ ]:
## Students practice

---

## Now we pull everything up and evaluate them a bit more systematically 

In [ ]:
Linear=linear_regression_make(feature_normaliser)
Linear.compile(optimizer=tf.optimizers.SGD(learning_rate=0.01),
    loss='mean_squared_error')

Model1 = keras.Sequential(name='One_layer_model')
Model1.add(feature_normaliser)
Model1.add(layers.Dense(16,activation='sigmoid',name='sigmoid'))
Model1.add(layers.Dense(1,name='dense_after_sigmoid'))

Model1.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

Model2 = keras.Sequential(name='Two_layers_model')
Model2.add(feature_normaliser)
Model2.add(layers.Dense(16,activation='sigmoid',name='sigmoid0'))
Model2.add(layers.Dense(8,name='linear_8X4'))
Model2.add(layers.Dense(4,activation='sigmoid',name='sigmoid1'))
Model2.add(layers.Dense(1,name='linear_8X1'))

Model2.summary()

Model2.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

history0 = Linear.fit(
    X_train,
    y_train,
    epochs=100,
    verbose=0,
    validation_data = (X_test,
                      y_test))

history1 = Model1.fit(
    X_train,
    y_train,
    epochs=100,
    verbose=0,
    validation_data = (X_test,
                      y_test))

history2 = Model2.fit(
    X_train,
    y_train,
    epochs=100,
    verbose=0,
    validation_data = (X_test,
                      y_test))

In [ ]:
plt.figure()
plt.plot(history0.history['loss'], label='Model 1 training loss',color='lightblue')
plt.plot(history0.history['val_loss'], label='Model 1 validation loss',color='blue')
plt.legend()
plt.grid(True)

plt.figure()
plt.plot(history1.history['loss'], label='Model 1 training loss',color='darkviolet')
plt.plot(history1.history['val_loss'], label='Model 1 validation loss',color='violet')
plt.legend()
plt.grid(True)

plt.figure()
plt.plot(history2.history['loss'], label='Model 2 training loss',color='darkgreen')
plt.plot(history2.history['val_loss'], label='Model 2 validation loss',color='lime')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
y_pred0=Linear.predict(X)
y_pred1=Model1.predict(X)
y_pred2=Model2.predict(X)

aligned_result=df(y).reset_index().join(df(y_pred0)).join(df(y_pred1),rsuffix='1').join(df(y_pred2),rsuffix='11').set_index('index')
# aligned_result.plot()
aligned_result.columns=['data','prediction_Linear','prediction_from_M1','prediction_from_M2']
aligned_result.sort_index().plot()

In [ ]:
print('Linear model: %.3f' % Linear.evaluate(X_test, y_test,verbose=0))
print('Model 1: %.3f' % Model1.evaluate(X_test, y_test,verbose=0))
print('Model 2: %.3f' % Model2.evaluate(X_test, y_test,verbose=0))

**Exercise:** what would happen if we run the models for another 100 steps (epochs)? What about another 300 stpes after that?